In [78]:
from typing import List, Dict, Tuple, Set
import numpy as np
import time
from datetime import datetime
from dateutil import parser
from dataclasses import dataclass
import time
from datetime import date
from collections import deque
import pandas as pd
from pathlib import Path
from enum import Enum
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from io import StringIO

# CHROME_WEBDRIVER_PATH = "/home/fabrice/Documents/PROGRAMMING/ChromeWebDrive/chromedriver"
CHROME_WEBDRIVER_PATH = "/usr/lib/chromium-browser/chromedriver"

# For retries
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Local files.
import constants as c
from constants import C_all as C

# %pip install selenium, html5lib

# Data Grabber

## Get an unprocessed url

In [79]:
df_competition_links = pd.read_csv("results/competition_links.csv")
df_competition_links

,Tournament link,Base url,Crawl date,Tournament id,Processed
0,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,9,0
1,https://ergebnisse.dancecomp.de/2022/28-0207_w...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,10,0
2,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,11,0
3,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,12,0
4,https://www.gruen-weiss-aachen.de/files/ergebn...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,13,0
...,...,...,...,...,...
218,https://www.hessen-tanzt.de/media/ht2023/56-13...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,227,0
219,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,228,0
220,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,229,0
221,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,230,0


In [80]:
print("URLs of already processed competitions:")
df_competition_links.loc[df_competition_links['Processed'] == 1]

URLs of already processed competitions:


,Tournament link,Base url,Crawl date,Tournament id,Processed


In [81]:
n = len(df_competition_links)
n_processed = len(df_competition_links.loc[df_competition_links['Processed'] == 1])
print(f"Nr. of crawled   competitions:  {n}\n"\
      f"Nr. of processed competitions:  {n_processed}\t({n_processed/n*100:0.2f} %)")

Nr. of crawled   competitions:  223
Nr. of processed competitions:  0	(0.00 %)


In [82]:
# Select the links and the ids of unprocessed tournaments.
unprocessed_tournaments_df = df_competition_links[df_competition_links['Processed'] == 0][[C.TOUR_LINK, C.ID]]

for competition_url, competition_id in unprocessed_tournaments_df.itertuples(index=False):
    print(f"Id: {competition_id}\tURL: {competition_url}")

Id: 9	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/3le-sen1sstd/index.htm
Id: 10	URL: https://ergebnisse.dancecomp.de/2022/28-0207_wdsfopenstdsen3/index.htm
Id: 11	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2022/dm_hgrsstd/index.htm
Id: 12	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/glinde-sen3sstd/index.htm
Id: 13	URL: https://www.gruen-weiss-aachen.de/files/ergebnisse/2023/2023-06-3le/3-1706_rl_hgrslat/index.htm
Id: 14	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/g55-2/index.htm
Id: 15	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/l66-4/index.htm
Id: 16	URL: https://berlin-dance-festival.de/files/bdf/results/2023/5-0104_wdsfopenstdjun2/index.htm
Id: 17	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2020/02-22.Deutsche.Meisterschaft.Junioren.II.B.Latein/index.htm
Id: 18	URL: https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/dpsen3slat/index.htm
Id: 19	URL: https://www.tanzsport.de/files/

## Crawl the data directly from the source code of the competition website

In [169]:
url_id = 1

Tested for url id indices:
- With 4 tabs - GER
    - 0
    - 4
- With 5 tabs - GER
    - 2
- With 6 tabs - GET
    - 3

In [170]:
is_english: bool = False
competition_url: str = unprocessed_tournaments_df[C.TOUR_LINK][url_id]
competition_id: int  = unprocessed_tournaments_df[C.ID][url_id]
# Start a Chrome driver to open the selected url.
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(competition_url)

# From the tournament link - extract the competition links.
# menu_url = re.compile('src="(.*)"').search(html).group(1)
competition_menue_links = [competition_url.replace('index.htm', "menu.htm#id2")]
df_tables = []
for l in competition_menue_links:
    driver.get(l)
    # Wait a second since the menu has to build itself before it can be collected.
    time.sleep(1)
        
    html: str = driver.page_source
    comp_date: str = None
    comp_class: str = None
    comp_title: str = None
    
    # Extract the title, class and date of the competition.
    # German format:    dd.mm.yyyy
    # English format:   dd/Mon/yyyy
    comp_date_and_class: str = driver.title
    match_date_de = re.search(r'\b(\d{2}\.\d{2}.\d{4})\b', comp_date_and_class)
    match_date_en = re.search(r'(\d{2}/[a-zA-Z]+/\d{4})', comp_date_and_class)
    if match_date_de:
        comp_date = match_date_de.group(1)
    elif match_date_en:
        comp_date = match_date_en.group(1)
        is_english = True
    comp_class = comp_date_and_class.replace(f"{comp_date} ", '')
    assert comp_date is not None, f"Unknown html! Could not extract competition date from {comp_date_and_class}."
    assert comp_class is not None, f"Unknown html! Could not extract competition class from {comp_date_and_class}."
    # Regardless what dateformat was extracted, save it in german format.
    parsed_date = parser.parse(comp_date)
    german_date_format = '%d.%m.%Y'
    comp_date = parsed_date.strftime(german_date_format)

    match_title = re.search(r'<td>(.*?)</td>', html)
    if match_title:
        comp_title = match_title.group(1)
    assert comp_title is not None, "Unknown html! Could not find competition name!"

    df_tables.append(pd.read_html(StringIO(html)))

# It may be that the data is stored in a nested DataFrame. 
# In this case extract it.
if len(df_tables) == 1:
    df_tables = df_tables[0]

# Since the download is complete now, close the driver.
driver.close()

print(competition_url)
df_tables

https://ergebnisse.dancecomp.de/2022/28-0207_wdsfopenstdsen3/index.htm


[                                                   0  1
 0  WDSF Open Standard Senior III und DTV-RL - Wup...  ≡,
                       0                                            1
 0            Organizer:                                          TNW
 1   Master of Ceremony:               Paaß, Patric TGC Rot-Weiß Porz
 2          Chairperson:     Sónyi, Markus Grün-Gold-Casino Wuppertal
 3         Adjudicators:                                          NaN
 4                    C:  Roßnagel, Bernd Schwarz-Weiß-Club Pforzheim
 5                    H:                  Ribas Lacroix, Diana France
 6                    K:                      Chojnacki, Marek Poland
 7                    M:                   Makarov, Aleksandr Estonia
 8                    O:                       Goetz, Hermann Austria
 9                    Q:                     Magnelli, Giuseppe Italy
 10                   R:                       Horacek, Petr Slovakia
 11                   T:                     

In [171]:
# TODO: If englisch: 
C = c.get_constants_in_language.get('de')
if is_english:
    C = c.get_constants_in_language.get('en')
    print("Is english!")
    # In this case skip the tournament since I expect it to be international (e.g. WDSF)
    # and thus with a lot of international adjudicators which will be underrepresented in the dataset.

Is english!


In [172]:
print(f"Title:\t{comp_title}")
print(f"Date:\t{comp_date}")
print(f"Name:\t{comp_class}")

Title:	WDSF Open Standard Senior III und DTV-RL - Wuppertal
Date:	02.07.2022
Name:	Sen.III Standard


In [173]:
# Create a folder to store the tables. Save a table and an equivalent README.md, containing the website link and the crawl date.
competition_dir = Path('results', 'comp_tables', f"{competition_id}")
# If directories do not exist, create them.
competition_dir.mkdir(parents=True, exist_ok=True)

# Save key information into a DataFrame and save it in the dir.
crawl_date = date.today().strftime("%B %d, %Y")
df_readme = pd.DataFrame({'Competition url': [competition_url],
                          'Crawl date': [crawl_date]})
df_readme.to_csv(Path(competition_dir, 'README.csv'))
# Save key information into a Markdown file and save it in the dir.
with open(Path(competition_dir, 'README.md'), 'w') as f:
    f.write(f'[Competition URL]({competition_url})\n')
    f.write(f'Crawled on {crawl_date}\n')
    f.write('by Fabrice Beaumont')

In [174]:
# Parse all tables.
# List of possible data tables:
class data_dfs(Enum):
    general_info: str   = 'General informations'
    adjudicators: str   = 'Adjudicators'
    ranking_list: str   = 'Ranking list'
    finals: str         = 'Finals'
    qualifications: str = 'Qualifications'

df_general_info: pd.DataFrame   = None
df_couples: pd.DataFrame        = None
df_finals: pd.DataFrame         = None
df_dict_raw: Dict[str, List[pd.DataFrame]] = {
    data_dfs.general_info: [],
    data_dfs.adjudicators: None,
    data_dfs.ranking_list: [],
    data_dfs.finals: [],
    data_dfs.qualifications: [],
}
df_dict: Dict[str, List[pd.DataFrame]] = {
    data_dfs.general_info: [],
    data_dfs.adjudicators: None,
    data_dfs.ranking_list: [],
    data_dfs.finals: [],
    data_dfs.qualifications: [],
}

# The first table is expected to be the competiton name in all cases.
competition_name = df_tables[0][0][0]

for i in range(1, len(df_tables)):
    table = df_tables[i]

    # Test for different criteria to sort the tables into the dictionary.
    # GENERAL INFORMATION table.
    if table[0][0] in C.get_organizer_keys():
        print(f"Found general info\t\tin table {i}")
        df_dict_raw[data_dfs.general_info].append(table)
        continue
    
    # RANKING LIST table - finals section.
    if len(table[0]) > 1 and str(table[0][0]) == C.KEY_FINAL:
        print(f"Found ranking list (finals)\tin table {i}")
        df_dict_raw[data_dfs.ranking_list].append(table)
        continue

    # RANKING LIST table - all qualification rounds sections.
    if len(table[0]) > 1 and str(table[0][1]).endswith(C.KEY_ROUND):
        print(f"Found ranking list (round)\tin table {i}")
        df_dict_raw[data_dfs.ranking_list].append(table)
        continue
    
    # FINALS table (containing the scating system and placements for each couple from each adjudicator).
    if len(table[0]) > 1 and str(table[1][0]) == C.KEY_ADJUDICATOR:
        print(f"Found finals scating\t\tin table {i}")
        df_dict_raw[data_dfs.finals].append(table)
        continue
    
    # QUALIFICATIONS table (containing points for all couples).
    if len(table[0]) > 1 and str(table[0][1]) == C.KEY_ADJUDICATOR:
        print(f"Found qualifications\t\tin table {i}")
        df_dict_raw[data_dfs.qualifications].append(table)
        continue

    print(f"Did not process table {i}!")

Found general info		in table 1
Did not process table 2!
Did not process table 3!
Found ranking list (finals)	in table 4
Did not process table 5!
Did not process table 6!
Did not process table 7!
Did not process table 8!
Did not process table 9!
Did not process table 10!
Did not process table 11!
Did not process table 12!
Did not process table 13!
Did not process table 14!
Did not process table 15!
Did not process table 16!
Did not process table 17!
Did not process table 18!
Did not process table 19!


## Processing of the general information and adjudicator list

In [175]:
df_general_info = df_dict_raw[data_dfs.general_info][0].copy()
# Split the gereral information table into the tournament information and the adjudicator list.
split_id = np.where(df_general_info.iloc[:, 1].isnull() == True)[0][0]
# Extract the remaining rows as information about the adjudicators.
adjudicator_names = list(df_general_info.iloc[split_id+1:, 1])
adjudicator_ids = [x[:1] for x in list(df_general_info.iloc[split_id+1:, 0])]
adjudicators = dict(zip(adjudicator_ids, adjudicator_names))

# Extract the data into individual variables.
df_general_info = pd.DataFrame(df_general_info.iloc[:split_id, :])
df_general_info.set_index([0])
df_general_info.columns = [C.CATEGORY, C.VALUE]
df_general_info = df_general_info[1:]
# Remove colon in the category column if present.
df_general_info[C.CATEGORY] = df_general_info[C.CATEGORY].str.replace(':', '')

# Add more data to the general info.
additional_info: Dict[str, str] = {}
additional_info['Date'] = comp_date
additional_info['Title'] = comp_title
additional_info['Class'] = comp_class

df_general_info

,Category,Value
1,Master of Ceremony,"Paaß, Patric TGC Rot-Weiß Porz"
2,Chairperson,"Sónyi, Markus Grün-Gold-Casino Wuppertal"


In [176]:
# Process the dict of adjudicators.
adjudicators_id_club: Dict[str, Tuple[Tuple[str, str], str]] = dict()
for adjudicator_id, adjudicator_and_club in adjudicators.items():
    ac = adjudicator_and_club.split(" ")
    adjudicator, club = tuple(ac[0:2]), " ".join(ac[2:])
    
    adjudicators_id_club[adjudicator_id] = tuple([adjudicator, club])

# Create a DataFrame to hold the adjudicator information.
df_adjudicators = pd.DataFrame(columns=[C.SURNAME, C.NAME, C.CLUB])
# Add adjudicators which are unknown (defined as surname, name, club triple).
for _, adjudicator_data in adjudicators_id_club.items():
    name, surname = adjudicator_data[0]
    name = name.replace(',', '')
    surname = surname.replace(',', '')
    club = adjudicator_data[1]
    
    # Check if adjudicator is already known:    
    x = df_adjudicators.loc[(df_adjudicators[C.SURNAME] == surname) & (df_adjudicators[C.NAME] == name) & (df_adjudicators[C.CLUB] == club)]
    # Only add, if not already known.
    if len(x) == 0:   
        new_row = pd.DataFrame({C.SURNAME: surname, C.NAME: name, C.CLUB: club}, index=[0])
        df_adjudicators = pd.concat([new_row,df_adjudicators.loc[:]]).reset_index(drop=True)

nr_adjudicators = len(df_adjudicators)

df_dict[data_dfs.adjudicators] = df_adjudicators
df_dict[data_dfs.adjudicators].to_csv(Path(competition_dir, c.filenames.adjudicators.value))
df_dict[data_dfs.adjudicators]

,Surname,Name,Club
0,Nick,Kremichenski,England
1,David,Carrillo,Spain
2,Luigi,Bodini,Georgia
3,Andriy,Polyvka,Ukraine
4,Petr,Horacek,Slovakia
5,Giuseppe,Magnelli,Italy
6,Hermann,Goetz,Austria
7,Aleksandr,Makarov,Estonia
8,Marek,Chojnacki,Poland
9,Lacroix,Ribas,Diana France


## Processing of the ranking list

In [177]:
df_dict_raw[data_dfs.ranking_list][0]

,0,1,2,3
0,Final,Final,Final,Final
1,Rank,No.,Couple,Country
2,1.,404,Slawek Lukawczyk / Janine-Nicole Desai,England
3,2.,438,Gert Faustmann / Alexandra Kley,Blau-Silber Berlin Tanzsportclub
4,3.,482,Vittorio Bramati / Silvia Martorelli,Italy
...,...,...,...,...
152,140.- 141.,844,Herwig Servotte / Betty Buelens,Belgium
153,142.,966,Rainer Stampfuß / Carola Käding,TSG Leverkusen
154,143.- 144.,497,Dr. Heinrich-Theodor Hannen / Dr. Beate Dittri...,TSC Grün-Gold Speyer
155,143.- 144.,698,Peter Mols / Annelies Mols,Netherlands


In [179]:
df_ranking_finals = df_dict_raw[data_dfs.ranking_list][0].copy()
# Drop the first row which contains only the key word 'Finals'.
df_ranking_finals = df_ranking_finals[1:]
# Set the first row as headder, and drop it form the data body.
df_ranking_finals.columns = df_ranking_finals.iloc[0]
df_ranking_finals = df_ranking_finals[1:]

# Since the placements of the finals are redundant information and will be extracted from the finals table,
# delete it here. In case that there were more rounds this allows to easily join the other rankings to this table.
df_ranking_finals.drop(columns=C.get_dancenames_short() + ['PZ'], inplace=True, errors='ignore')
# Split the club name from the couples name (and their number).
df_ranking_finals[[C.COUPLE, C.CLUB]] = df_ranking_finals[C.KEY_COUPLE].str.split(")", expand = True)
df_ranking_finals.drop(columns=[C.KEY_COUPLE], inplace=True)
df_ranking_finals[[C.COUPLE, C.NR]] = df_ranking_finals[C.COUPLE].str.split("(", expand = True)
df_ranking_finals[[C.ROUND]] = C.KEY_FINAL
df_ranking_finals = df_ranking_finals[[C.PLACEMENT, C.ROUND, C.NR, C.COUPLE, C.CLUB]]
df_ranking_finals

AttributeError: 'C_en' object has no attribute 'KEY_COUPLE'

In [ ]:
df_ranking_other = df_dict_raw[data_dfs.ranking_list][1].copy()
# Drop all rows which contain only NaN values. These are originally used to separate the tables.
df_ranking_other.dropna(axis=0, how='all', inplace=True)
# Rename the columns.
df_ranking_other.columns = [C.PLACEMENT, C.COUPLE, C.CLUB]

# Split the club name from the couples name (and their number).
df_ranking_other[[C.NR]]     = df_ranking_other[C.COUPLE].str.extract(r'\((\d*)\)', expand = True)
df_ranking_other[[C.COUPLE]] = df_ranking_other[C.COUPLE].str.extract(r'(.*)\(', expand = True)

# Collect the row indices that contain names instead of placements. These are the round names.
df_ranking_other.reset_index(inplace=True)
round_rows = df_ranking_other[df_ranking_other[C.PLACEMENT].str.endswith(C.KEY_ROUND)]
nr_rounds: int = len(round_rows)
round_list: List[Tuple[int, str]] = [(i, round_rows[C.PLACEMENT][i]) for i in round_rows.index] + [(len(df_ranking_other), '')]

# Mark all following rows with the corresponding round name in a new column.
df_ranking_other[[C.ROUND]] = round_list[0][1]
for i in range(len(round_list) - 1):
    start_index, round_name = round_list[i]
    end_index = round_list[i + 1][0]
    df_ranking_other.iloc[start_index : end_index, df_ranking_other.columns.get_loc(C.ROUND)] = round_name


# Now delete the rows with the round names. 
# Notice that we ignore the last entry since this is an artificial one to finish the for-loop above.
for i, _ in round_list[:-1]:
    df_ranking_other.drop(i, inplace=True)

# Rearrange the column order.
df_ranking_other = df_ranking_other[[C.PLACEMENT, C.ROUND, C.NR, C.COUPLE, C.CLUB]]

df_ranking_other

,Platz,Runde,Nr.,Paar,Club
1,7.,Vorrunde,121,"Darii, Maksym / Kondratenko, Sophie",TSC Rot-Gold-Casino Nürnberg
2,8.,Vorrunde,77,"Ortiz García, Carlos / Ortiz García, Christina",TTC Rot-Gold Köln
3,9.,Vorrunde,103,"Lindner, Nico / Klemm, Tabea",Schwarz-Weiß-Club Pforzheim
4,10.,Vorrunde,75,"Werner, Leo / Theobalt, Fabienne",TC Rot-Weiss Casino Mainz
5,11.,Vorrunde,31,"Falke, Leon / Shishkina, Anastasia",Grün-Gold-Club Bremen
6,12.,Vorrunde,72,"Yerokhin, Nikita / Muhamedagic, Ela",Grün-Gold-Club Bremen


In [ ]:
df_rankings = pd.concat([df_ranking_finals, df_ranking_other])
# Add two more columns for the separated names of the leader and the follower.
df_rankings[[C.LEADER, C.FOLLOWER]] = df_rankings[C.COUPLE].str.split(" / ", expand = True)
nr_couples: int = len(df_rankings)
# Set the couples number as row index.
df_rankings.set_index(C.NR, inplace=True)
df_dict[data_dfs.ranking_list] = df_rankings
df_dict[data_dfs.ranking_list].to_csv(Path(competition_dir, c.filenames.ranking_lst.value))
df_dict[data_dfs.ranking_list]

,Platz,Runde,Paar,Club,Leader,Follower
Nr.,,,,,,
33,1.,Endrunde,"Jenner, David / Tuigunov, Elisabeth",Die Residenz Münster,"Jenner, David","Tuigunov, Elisabeth"
104,2.,Endrunde,"Schmuck, Daniel / Tudorache, Irina",TSC Rot-Gold-Casino Nürnberg,"Schmuck, Daniel","Tudorache, Irina"
92,3.,Endrunde,"Elert, Justin / Menser, Nicole",Rot-Weiss-Klub Kassel,"Elert, Justin","Menser, Nicole"
98,4.,Endrunde,"Pastuchow, Daniel / Elkin, Anastasia",Blau-Silber Berlin Tanzsportclub,"Pastuchow, Daniel","Elkin, Anastasia"
117,5.,Endrunde,"Anisimov, Demid / Domingues da Silva, Giuliana",Grün-Gold-Club Bremen,"Anisimov, Demid","Domingues da Silva, Giuliana"
76,6.,Endrunde,"Deiloff, Dennis / Deiloff, Sabrina",TGC Rot-Weiß Porz,"Deiloff, Dennis","Deiloff, Sabrina"
121,7.,Vorrunde,"Darii, Maksym / Kondratenko, Sophie",TSC Rot-Gold-Casino Nürnberg,"Darii, Maksym","Kondratenko, Sophie"
77,8.,Vorrunde,"Ortiz García, Carlos / Ortiz García, Christina",TTC Rot-Gold Köln,"Ortiz García, Carlos","Ortiz García, Christina"
103,9.,Vorrunde,"Lindner, Nico / Klemm, Tabea",Schwarz-Weiß-Club Pforzheim,"Lindner, Nico","Klemm, Tabea"


## Processing of the finals

In [ ]:
df_finals = df_dict_raw[data_dfs.finals][0]
dance_name_rows = df_finals[df_finals[0].isna()]
dance_name_ids = dance_name_rows.index.to_list() + [len(df_finals.index)]

# Add a column to display the dance for which the rankings are for. 
# This will replace the horizonal headlines indicating the dances - which will be deleted afterwards.
df_finals[[C.DANCE]] = ''
df_finals[C.DANCE].iloc[1] = C.DANCE
for i in range(len(dance_name_ids) - 1):
    start_index = dance_name_ids[i]
    dance_name = c.parse_dance_name(dance_name_rows.loc[start_index, 1], C)
    end_index = dance_name_ids[i+1] - 1
    df_finals.loc[start_index:end_index, C.DANCE] = dance_name

# Now delete the rows with the dance names. 
# Notice that we ignore the last entry since this is an artificial one to finish the for-loop above.
for i in dance_name_ids[:-1]:
    df_finals.drop(i, inplace=True)

# Replace the couple names by only their number.
df_finals[[C.NR]] = df_finals.iloc[:, 0].str.extract(r'(\d+)\D*', expand = True)
df_finals[C.NR].iloc[1] = C.NR
df_finals = df_finals.iloc[:, 1:]

# Replace the adjudicator names by only their reference letter.
for i, value in enumerate(df_finals.iloc[1, :]):
    if value == "1.": 
        break

    df_finals.iloc[1, i] = value[:1]

# Delete the first line which only denotes where the adjudicators and results columns are - same as the second row does.
df_finals = df_finals.iloc[1:]
# Set the now first row as header.
df_finals.columns = df_finals.iloc[0]
df_finals = df_finals[1:]

# Use the couple number as index column.
df_finals.set_index(C.NR, drop=True, inplace=True)
df_finals.index.name = C.NR

df_dict[data_dfs.finals] = df_finals
df_dict[data_dfs.finals].to_csv(Path(competition_dir, c.filenames.finals.value))
df_dict[data_dfs.finals]

1,A,B,C,D,E,G,J,1.,1.-2.,1.-3.,1.-4.,1.-5.,1.-6.,Platz,Summe,Dance
Nr.,,,,,,,,,,,,,,,,
33,1,1,1,1,1,1,2,6,-,-,-,-,-,10,10,SB
76,4,6,3,6,4,5,6,0,0,1,3,4(16),-,60,60,SB
92,3,4,2,4,2,4,5,0,2,3,6,-,-,30,30,SB
98,6,3,5,3,6,6,4,0,0,2,3,4(15),-,50,50,SB
104,2,2,4,2,3,3,1,1,4,-,-,-,-,20,20,SB
117,5,5,6,5,5,2,3,0,1,2,2,6,-,40,40,SB
33,1,1,1,1,1,1,2,6,-,-,-,-,-,10,20,CC
76,4,6,3,6,4,5,6,0,0,1,3,4,-,60,120,CC
92,3,4,2,4,2,4,5,0,2,3,6,-,-,30,60,CC


## Processing of the qualifications table

In [ ]:
row_id_marks: int = 0
row_id_sums: int  = 1
row_id_quali: int = 2
row_id_round_offset: int = 3

In [ ]:
df_qualifications = df_dict_raw[data_dfs.qualifications][0]
n: int = len(df_dict_raw[data_dfs.qualifications])
if n > 1:
    for i in range(1, n):
        # Drop the first column since it is already present from the first dataframe.
        df_tmp = df_dict_raw[data_dfs.qualifications][i].iloc[:, 1:]
        # Append the dataframe to the rest.
        df_qualifications = pd.concat([df_qualifications, df_tmp], axis=1)

# Delete the first row since it does not really contain information
df_qualifications = df_qualifications[1:]
# Because of the concatenation the column indices could be messed up. Reset them to a range.
df_qualifications.columns = range(len(df_qualifications.columns))

# Delete all rows where all values are NaN.
df_qualifications.dropna(axis=1, how='all', inplace=True)

# Depending on the size of the tournament, the names of the competitors are perhaps repeated.
# In this case frop this row since it is redundant.
df_qualifications.drop(df_qualifications.loc[df_qualifications.iloc[:, 0] == 'Startnummer'].index, inplace=True)


# Replace the adjudicator names by only their reference letter.
# Replace the couple names by only their number.
couple_nrs = df_qualifications.iloc[0, :].str.extract(r'(\d+)\D*')
couple_nrs_list = couple_nrs[0].tolist()
couple_nrs_list[0] = C.NR
df_qualifications.columns = couple_nrs_list
# Drop the first row since it contains only the names of the couples and is thus 
# equivalent to the new column names.
df_qualifications = df_qualifications[1:]
df_qualifications

/tmp/ipykernel_92588/2879684803.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qualifications.dropna(axis=1, how='all', inplace=True)
/tmp/ipykernel_92588/2879684803.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qualifications.drop(df_qualifications.loc[df_qualifications.iloc[:, 0] == 'Startnummer'].index, inplace=True)


,Nr.,31,33,72,75,76,77,92,98,103,104,117,121
2,"A) Becker, Werner B) Hoffmann, Gabor-Istvan C)...",0 0 1 0 0 0 0,5 5 5 5 5 5 5,0 0 0 0 0 0 0,0 0 0 0 0 4 0,5 0 5 0 5 4 0,1 4 0 0 4 0 0,5 5 5 5 1 5 5,5 5 4 5 2 3 5,0 1 0 0 4 0 0,5 5 5 5 4 3 5,4 5 5 5 5 5 5,0 0 0 5 0 1 5
3,Ergebnis der Vorrunde,1,35,0,4,19,9,31,29,5,32,34,11
4,Qualifiziert für die Endrunde,NaN,X,NaN,NaN,X,NaN,X,X,NaN,X,X,NaN
5,Addition Endrunde,NaN,50,NaN,NaN,300,NaN,150,210,NaN,100,240,NaN
7,Platz von Platz bis,11,1,12,10,6,8,3,4,9,2,5,7
8,Aufstiegspunkte,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
for r in range(nr_rounds):
    # Split the values and map them to new columns. These correspond to the marks of the adjudicaotrs in order.
    round_marks = df_qualifications.iloc[row_id_marks + r * row_id_round_offset, 1:]
    round_marks.replace(float('Nan'), str('- ' * nr_adjudicators)[:-1], inplace=True)
    df_round_marks = round_marks.str.split(' ', expand=True)
    df_round_marks.columns = adjudicator_ids
    df_round_marks[C.NR] = list(df_qualifications.columns)[1:]
    df_round_marks.set_index(C.NR, drop=True, inplace=True)
    df_round_marks.index.name = C.NR

    # Etract the sums.
    sums = df_qualifications.iloc[row_id_sums + r * row_id_round_offset, 1:]
    sums.replace(float('NaN'), -1, inplace=True)
    df_round_marks[C.SUM] = sums

    # Extract the qualification indications, and replace the values by binary ones.
    quali = df_qualifications.iloc[row_id_quali + r * row_id_round_offset, 1:]
    quali.replace(float('NaN'), False, inplace=True)
    quali.replace('X', True, inplace=True)
    # Add the sums as column to the DataFrame.
    df_round_marks[C.QUALI] = quali

    df_dict[data_dfs.qualifications].append(df_round_marks)
    df_round_marks.to_csv(Path(competition_dir, c.filenames.quali_roundx.value.format(r)))

In [ ]:
df_dict[data_dfs.qualifications][0]

,A,B,C,D,E,G,J,Sum,Qualified
Nr.,,,,,,,,,
31,0,0,1,0,0,0,0,1,False
33,5,5,5,5,5,5,5,35,True
72,0,0,0,0,0,0,0,0,False
75,0,0,0,0,0,4,0,4,False
76,5,0,5,0,5,4,0,19,True
77,1,4,0,0,4,0,0,9,False
92,5,5,5,5,1,5,5,31,True
98,5,5,4,5,2,3,5,29,True
103,0,1,0,0,4,0,0,5,False


In [ ]:
df_dict[data_dfs.qualifications][-1]

,A,B,C,D,E,G,J,Sum,Qualified
Nr.,,,,,,,,,
31,0,0,1,0,0,0,0,1,False
33,5,5,5,5,5,5,5,35,True
72,0,0,0,0,0,0,0,0,False
75,0,0,0,0,0,4,0,4,False
76,5,0,5,0,5,4,0,19,True
77,1,4,0,0,4,0,0,9,False
92,5,5,5,5,1,5,5,31,True
98,5,5,4,5,2,3,5,29,True
103,0,1,0,0,4,0,0,5,False


## Add further collected information to the general information table and save it

In [ ]:
additional_info[C.NR_ROUNDS] = nr_rounds + 1
additional_info[C.NR_ADJDCTRS] = nr_adjudicators
additional_info[C.NR_COUPLES] = nr_couples
df_additional_info = pd.DataFrame(list(additional_info.items()), columns=[C.CATEGORY, C.VALUE])

df_dict[data_dfs.general_info] = pd.concat([df_general_info, df_additional_info])
df_dict[data_dfs.general_info].to_csv(Path(competition_dir, c.filenames.general_info.value))
df_dict[data_dfs.general_info]

,Category,Value
1,Ausrichter,TSC Grün-Weiß Aquisgrana Aachen
2,Turnierleiter,"Pitz, Marcel TSC Grün-Weiß Aquisgrana Aachen"
3,Beisitzer,"Belten, Markus TSC Grün-Weiß Aquisgrana Aachen"
4,Protokoll,"Belten, Ursula TSC Grün-Weiß Aquisgrana Aachen"
5,Turnierbüro,"Angelina TSC Grün-Weiß Aquisgrana Aachen Brix,..."
6,Musik,"Hock, Tobias TSC Grün-Weiß Aquisgrana Aachen L..."
0,Date,17.06.2023
1,Title,RL Hgr S Lat - Aachen
2,Class,Hgr. S Latein
3,Nr. Rounds + Final,2


In [ ]:
# Test if the parsing has been sucessfull, that is all desired dataframes contain data.
all(len(x) > 0 for x in df_dict.values())

True